In [2]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string
from google.colab import drive
import torch
from keras_preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
from tqdm import tqdm

nltk.download('punkt') # if necessary...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pd.set_option('display.max_colwidth', None)
data_path = '/content/gdrive/MyDrive/SimilarityAnalyzer/combined.csv'
df = pd.read_csv(data_path, index_col = 0)
print(df.shape)
df.head(3)

(49801, 1)


,abstract
0,"Transformers have shown great potential in computer vision tasks. A common\nbelief is their attention-based token mixer module contributes most to their\ncompetence. However, recent works show the attention-based module in\ntransformers can be replaced by spatial MLPs and the resulted models still\nperform quite well. Based on this observation, we hypothesize that the general\narchitecture of the transformers, instead of the specific token mixer module,\nis more essential to the model's performance. To verify this, we deliberately\nreplace the attention module in transformers with an embarrassingly simple\nspatial pooling operator to conduct only the most basic token mixing.\nSurprisingly, we observe that the derived model, termed as PoolFormer, achieves\ncompetitive performance on multiple computer vision tasks. For example, on\nImageNet-1K, PoolFormer achieves 82.1% top-1 accuracy, surpassing well-tuned\nvision transformer/MLP-like baselines DeiT-B/ResMLP-B24 by 0.3%/1.1% accuracy\nwith 35%/52% fewer parameters and 48%/60% fewer MACs. The effectiveness of\nPoolFormer verifies our hypothesis and urges us to initiate the concept of\n""MetaFormer"", a general architecture abstracted from transformers without\nspecifying the token mixer. Based on the extensive experiments, we argue that\nMetaFormer is the key player in achieving superior results for recent\ntransformer and MLP-like models on vision tasks. This work calls for more\nfuture research dedicated to improving MetaFormer instead of focusing on the\ntoken mixer modules. Additionally, our proposed PoolFormer could serve as a\nstarting baseline for future MetaFormer architecture design. Code is available\nat https://github.com/sail-sg/poolformer"
1,"A critical aspect of reliable communication involves the design of codes that\nallow transmissions to be robustly and computationally efficiently decoded\nunder noisy conditions. Advances in the design of reliable codes have been\ndriven by coding theory and have been sporadic. Recently, it is shown that\nchannel codes that are comparable to modern codes can be learned solely via\ndeep learning. In particular, Turbo Autoencoder (TURBOAE), introduced by Jiang\net al., is shown to achieve the reliability of Turbo codes for Additive White\nGaussian Noise channels. In this paper, we focus on applying the idea of\nTURBOAE to various practical channels, such as fading channels and chirp noise\nchannels. We introduce TURBOAE-TI, a novel neural architecture that combines\nTURBOAE with a trainable interleaver design. We develop a carefully-designed\ntraining procedure and a novel interleaver penalty function that are crucial in\nlearning the interleaver and TURBOAE jointly. We demonstrate that TURBOAE-TI\noutperforms TURBOAE and LTE Turbo codes for several channels of interest. We\nalso provide interpretation analysis to better understand TURBOAE-TI."
2,"Point defects are responsible for a wide range of optoelectronic properties\nin materials, making it crucial to engineer their concentrations for novel\nmaterials design. However, considering the plethora of defects in co-doped\nsemiconducting and dielectric materials and the dependence of defect formation\nenergies on heat treatment parameters, process design based on an experimental\ntrial and error approach is not an efficient strategy. This makes it necessary\nto explore computational pathways for predicting defect equilibria during heat\ntreatments. The accumulated experimental knowledge on defect transformations in\ndiamond is unparalleled. Therefore, diamond is an excellent material for\nbenchmarking computational approaches. By considering nitrogen, hydrogen, and\nsilicon doped diamond as a model system, we have investigated the pressure\ndependence of defect formation energies and calculated the defect equilibria\nduring heat treatment of diamond through ab-initio calculations. We have\nplotted monolithic-Kr\""oger-Vink diagrams for various defects, representing\nd

In [ ]:
def preprocess_data(data_path, sample_size):

  # Read the data from specific path
  data = pd.read_csv(data_path, index_col=0)
  # data = pd.read_csv(data_path, low_memory=False, quoting=3, error_bad_lines=False)

  # Drop articles without Abstract
  data = data.dropna(subset = ['abstract']).reset_index(drop = True)

  # Get "sample_size" random articles
  data = data.sample(sample_size)[['abstract']]

  return data
  
source_data = preprocess_data(data_path, 20000)
print(source_data.shape)
source_data.head()


(20000, 1)


,abstract
34322,"Tularemia is a severe infectious zoonotic disease caused by Francisella tularensis. Although F. tularensis is considered to be a potential biological weapon due to its high infectivity and mortality rate, no vaccine has been currently licensed. Recently, we reported that F. tularensis SCHU P9 derived ΔpdpC strain lacking the pathogenicity determinant protein C gene conferred stable and good protection in a mouse lethal model. In this study, the protective effect of ΔpdpC was evaluated using a monkey lethal model. Two cynomolgus macaques (Macaca fascicularis) intratracheally challenged with the virulent strain SCHU P9 were euthanized on 7 and 11 days post-challenge after the development of severe clinical signs. The bacterial replication in alveolar macrophages and type II epithelial cells in the lungs would cause severe pneumonia accompanied by necrosis. Conversely, two animals subcutaneously immunized with ΔpdpC survived 3 weeks after SCHU P9 challenge. Though one of the two animals developed mild symptoms of tularemia, bacterial replication was limited in the respiratory organs, which may be due to a high level of humoral and cellular immune responses against F. tularensis. These results suggest that the ΔpdpC mutant would be a safe and promising candidate as a live attenuated tularemia vaccine."
5675,"Identifying covariate shift is crucial for making machine learning systems\nrobust in the real world and for detecting training data biases that are not\nreflected in test data. However, detecting covariate shift is challenging,\nespecially when the data consists of high-dimensional images, and when multiple\ntypes of localized covariate shift affect different subspaces of the data.\nAlthough automated techniques can be used to detect the existence of covariate\nshift, our goal is to help human users characterize the extent of covariate\nshift in large image datasets with interfaces that seamlessly integrate\ninformation obtained from the detection algorithms. In this paper, we design\nand evaluate a new visual interface that facilitates the comparison of the\nlocal distributions of training and test data. We conduct a quantitative user\nstudy on multi-attribute facial data to compare two different learned\nlow-dimensional latent representations (pretrained ImageNet CNN vs. density\nratio) and two user analytic workflows (nearest-neighbor vs.\ncluster-to-cluster). Our results indicate that the latent representation of our\ndensity ratio model, combined with a nearest-neighbor comparison, is the most\neffective at helping humans identify covariate shift."
46991,"Policies and public health efforts have not addressed the gendered impacts of disease outbreaks.1 The response to coronavirus disease 2019 (COVID-19) appears no different. We are not aware of any gender analysis of the outbreak by global health institutions or governments in affected countries or in preparedness phases. Recognising the extent to which disease outbreaks affect women and men differently is a fundamental step to understanding the primary and secondary effects of a health emergency on different individuals and communities, and for creating effective, equitable policies and interventions. Although sex-disaggregated data for COVID-19 show equal numbers of cases between men and women so far, there seem to be sex differences in mortality and vulnerability to the disease.2 Emerging evidence suggests that more men than women are dying, potentially due to sex-based immunological3 or gendered differences, such as patterns and prevalence of smoking.4 However, current sex-disaggregated data are incomplete, cautioning against early assumptions. Simultaneously, data from the State Council Information Office in China suggest that more than 90% of health-care workers in Hubei province are women, emphasising the gendered nature of the health workforce and the risk that predominantly female health workers incur."
12588,"a voting process was formalized as t

In [4]:
model = TfidfVectorizer()
model_path = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=True)

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))
    
def process_document(text):
    """
    Create a vector for given text and adjust it for cosine similarity search
    """
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect

def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):
    
    input_ids = tokenizer.encode(
                        text, 
                        add_special_tokens = True, 
                        max_length = MAX_LEN,                           
                   )    

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks    
    attention_mask = [int(i>0) for i in input_ids]
    
    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():        
        logits, encoded_layers = model(
                                    input_ids = input_ids, 
                                    token_type_ids = None, 
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]
        
    # Extract the embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vector = vector.detach().cpu().numpy()

    return(vector)

def create_vector_index(data):
    
    # The list of all the vectors
    vectors = []
    
    # Get overall text data
    source_data = data.abstract.values
    
    # Loop over all the comment and get the embeddings
    for text in tqdm(source_data):
        
        # Get the embedding 
        vector = create_vector_from_text(tokenizer, model, text)
        
        #add it to the list
        vectors.append(vector)
    
    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))
    
    return data

In [ ]:
vector_index = create_vector_index(source_data)